In [1]:
import os
import numpy as np
from finlab import data, backtest

current_path = os.path.dirname(os.getcwd())
import finlab
finlab.login(open(f"{current_path}/config.txt", "r").read())

import _beowFmt as fm 
import pandas as pd
from datetime import date

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 忽略FutureWarning

close =  data.get('price:收盤價')
volume = data.get('price:成交股數')
revenue = data.get('monthly_revenue:當月營收')

# print(revenue)
# 營收成長
c1 = (revenue.pct_change().average(6) > 0).index_str_to_date()

# 通道狹窄
c2 = close.rolling(20).min() > close.rolling(20).max() * 0.7

# 成交量與價格收縮
volumeReduce = volume.average(10) < (volume.average(60)*0.5)
priceContract = close.rolling(10).std() < (close.rolling(60).std()*0.5)

# VCP = 價量收縮、創新高、有足夠成交量
vcp = (volumeReduce & priceContract).sustain(5, 1) & (close == close.rolling(100).max()) & (volume >= volume.rolling(20).mean()*0.8)

# 股價低於150
c3 = revenue.average(5) > 100000  # 單位:千

# 總體部位
# 進場：(c1, c2, vcp, 流動性足夠, 長期均向上)
# 出場：股價 < 季線
buy = (vcp & c1 & c2 & c3 & (volume*close > 2000000) & (close > close.average(250)))  #長期均線250天
sell = close < close.average(60)

position = buy.hold_until(sell, nstocks_limit=20, rank=(revenue / revenue.average(3)).replace([np.inf, -np.inf], np.nan).index_str_to_date())

dff = position.iloc[-1:].transpose()
dff["stock_ids"] = dff.index
dff.columns = ["val", "stock_ids"]

dff = dff[ (dff["stock_ids"].str.len() == 4) & (dff["val"] == True) ]
dff = dff.dropna(how='all')

print(dff)
today, ss = date.today().strftime("%Y%m%d"), "",

for c in dff["stock_ids"].tolist():
    ss += f"{c}.TW,"
fm.write_LogFile(f"xq-VCP/{today}.csv", ss) 

# 過濾 KY股
# sc = data.get("security_categories")
# position_col = position.columns
# ky_filter = position_col[~position_col.isin(list(sc[sc["name"].str.contains("KY")]["stock_id"]))]
# position = position[ky_filter]


# 回測
r = backtest.sim(position, fee_ratio=1.425/1000/3, resample_offset='D', position_limit=0.2, resample='D', stop_loss=0.1)

輸入成功!


Your version is 1.1.1, please install a newer version.
Use "pip install finlab==1.1.3" to update the latest version.


Daily usage: 19.6 / 5000 MB - monthly_revenue:當月營收
         val stock_ids
symbol                
1436    True      1436
2348    True      2348
2906    True      2906
3171    True      3171
3303    True      3303
3312    True      3312
3356    True      3356
3622    True      3622
4113    True      4113
4571    True      4571
4933    True      4933
5264    True      5264
6190    True      6190
6221    True      6221
6442    True      6442
6523    True      6523
6577    True      6577
6761    True      6761
6863    True      6863
8401    True      8401
Daily usage: 23.3 / 5000 MB - etl:is_flagged_stock
